# Building, Training, and Deploying a Machine Learning Model with Amazon SageMaker

<br>

**Table of Contents:**

- [Libraries](#Libraries)
- [Defining the IAM Role](#Defining-the-IAM-Role)
- [Creating an S3 Bucket](#Creating-an-S3-Bucket)
- [Retrieving the Dataset](#Retrieving-the-Dataset)
- [Splitting into Training/Test Sets](#Splitting-into-Training/Test-Sets)
- [Training the Model](#Training-the-Model)
- [Deploying the Model](#Deploying-the-Model)
- [Evaluating Model Performance](#Evaluating-Model-Performance)
- [Cleaning up](#Cleaning-up)
    - [Deleting Endpoint](#Deleting-Endpoint)
    - [Deleting Training Artifacts and S3 Bucket](#Deleting-Training-Artifacts-and-S3-Bucket)

<br>

## Libraries

In [1]:
import boto3, sagemaker, re, os, urllib.request

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

from sagemaker import get_execution_role
from sagemaker.serializers import CSVSerializer

<br>

## Defining the IAM Role

In [2]:
# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name  # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region.")
print("You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-1 region.
You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


<br>

## Creating an S3 Bucket

In [3]:
bucket_name = 'sagemaker-tutorial-12345'  # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')

try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': my_region})
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ', e)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


S3 bucket created successfully


<br>

## Retrieving the Dataset

In [4]:
dataset_url = "https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv"

try:
    urllib.request.urlretrieve(url=dataset_url, filename="bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ', e)

try:
    model_data = pd.read_csv('./bank_clean.csv', index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ', e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


<br>

## Splitting into Training/Test Sets

In [5]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729),
                                 [int(0.7 * len(model_data))])

print('Train Set: {} rows x {} columns'.format(train_data.shape[0], train_data.shape[1]))
print(' Test Set: {} rows x {} columns'.format(test_data.shape[0], test_data.shape[1]))

(28831, 61) (12357, 61)


<br>

## Training the Model

In [6]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [7]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,
                                    role,
                                    instance_count=1,
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket_name, prefix),
                                    sagemaker_session=sess)

xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

In [8]:
xgb.fit({'train': s3_input_train})

2022-06-01 17:12:47 Starting - Starting the training job...ProfilerReport-1654103567: InProgress
...
2022-06-01 17:13:35 Starting - Preparing the instances for training.........
2022-06-01 17:15:05 Downloading - Downloading input data......
2022-06-01 17:16:01 Training - Downloading the training image...
2022-06-01 17:16:47 Training - Training image download completed. Training in progress..Arguments: train
[2022-06-01:17:16:51:INFO] Running standalone xgboost training.
[2022-06-01:17:16:51:INFO] Path /opt/ml/input/data/validation does not exist!
[2022-06-01:17:16:51:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8461.19mb
[2022-06-01:17:16:51:INFO] Determined delimiter of CSV input is ','
[17:16:51] S3DistributionType set as FullyReplicated
[17:16:51] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[17:16:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra no

<br>

## Deploying the Model

In [9]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

------------!

In [10]:
test_data_array = test_data.drop(labels=['y_no', 'y_yes'],
                                 axis=1).values  #load the data into an array

xgb_predictor.serializer = CSVSerializer()  # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')  # predict!
predictions_array = np.fromstring(predictions[1:], sep=',')  # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


<br>

## Evaluating Model Performance

In [11]:
cm = pd.crosstab(index=test_data['y_yes'],
                 columns=np.round(predictions_array),
                 rownames=['Observed'],
                 colnames=['Predicted'])
tn = cm.iloc[0, 0]
fn = cm.iloc[1, 0]
tp = cm.iloc[1, 1]
fp = cm.iloc[0, 1]
p = (tp + tn) / (tp + tn + fp + fn) * 100

print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format(
    "No Purchase", tn / (tn + fn) * 100, tn, fp / (tp + fp) * 100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format(
    "Purchase", fn / (tn + fn) * 100, fn, tp / (tp + fp) * 100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10769)    37% (167)
Purchase        10% (1133)     63% (288) 



<br>

## Cleaning up

### Deleting Endpoint

In [12]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

<br>

### Deleting Training Artifacts and S3 Bucket

In [13]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


[{'ResponseMetadata': {'RequestId': 'JYMGDHV3DT8MMSKZ',
   'HostId': 's7v06Y42SwzNT0UlCCT+rJWZA90JpofEvnbIwEaZyuf2+6H58MbX8ulKn5eN4DZL80W2+InrdqI=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 's7v06Y42SwzNT0UlCCT+rJWZA90JpofEvnbIwEaZyuf2+6H58MbX8ulKn5eN4DZL80W2+InrdqI=',
    'x-amz-request-id': 'JYMGDHV3DT8MMSKZ',
    'date': 'Wed, 01 Jun 2022 17:27:15 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2022-06-01-17-12-47-051/profiler-output/system/incremental/2022060117/1654103700.algo-1.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2022-06-01-17-12-47-051/rule-output/ProfilerReport-1654103567/profiler-output/profiler-reports/CPUBottleneck.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2022-06-01-17-12-47-051/rule-output/ProfilerReport-1654103567/profiler-output/profiler-

<br>

---